In [ ]:
#===============================================================================================
# Huggingface와 ONNX 런타임을 이용한 동적 양자화 예시
# => 훈련된 sentencebert 모델을 가지고, ONNX 모데로 만든 후, 테스트 하는 예시
# => 문장임베딩이므로, ORTModelForFeatureExtraction 이용
#
#
## 참고 : https://huggingface.co/docs/optimum/index
##        https://github.com/huggingface/optimum
##        https://huggingface.co/blog/optimum-inference
##
## 아래 패키지들을 설치해야 함
'''
!pip install datasets
!pip install optimum
!pip install optimum[onnxruntime]
!pip install optimum[onnxruntime-gpu]  #gpu 사용인 경우

'''    
#===============================================================================================

import numpy as np
import pandas as pd
import torch
import os
import torch.nn.functional as F
import sys

from myutils import seed_everything, GPU_info, mlogging
logger = mlogging(loggername="distilbertnlitest", logfilename="distilbertnlitest")
seed_everything(111)

In [ ]:
#============================================================
# 기존 bert 모델을 동적 양자화 시킴
#============================================================

from optimum.onnxruntime import ORTConfig, ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig

# 기존 bert 모델 경로 
model_checkpoint = "./sentencebert_v1.0"

# 동적 양자화인 경우 is_static = False로 해야 함.
qconfig = AutoQuantizationConfig.arm64(is_static=False, per_channel=False)

# 분류 모델인 경우에는 feature="sequence-classification"
quantizer = ORTQuantizer.from_pretrained(model_checkpoint, feature="default")

# ONNX 모델로 만들고 양자화 함
quantizer.export(
    onnx_model_path="Smodel.onnx",   # ONNX 모델 출력 경로
    # onnx 양자화 모델이 생성되는 경로(이름은 model.onnx로 해야함=>그래야 huggingface 함수 이용시 경로만 지정해도 자동으로 불어옴)
    onnx_quantized_model_output_path="./sentencebert_v1.0/model.onnx",  
    quantization_config=qconfig,
)

In [ ]:
# onnx 모델 구조 로딩 해봄.
# => 맨 뒤에 return %last_hidden_state 리턴되면 => ORTModelForFeatureExtraction 사용 가능
import onnx
model = onnx.load("./sentencebert_v1.0/model.onnx")
onnx.checker.check_model(model)
print(onnx.helper.printable_graph(model.graph))

In [ ]:
#============================================================
# ONNX 양자화 모델 불러옴
# => 문장임베딩이므로, ORTModelForFeatureExtraction 로 로딩함
#============================================================
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForFeatureExtraction, ORTModelForSequenceClassification

vocab_path = "./sentencebert_v1.0"
model_path = "./sentencebert_v1.0"

tokenizer = AutoTokenizer.from_pretrained(vocab_path)

# 문장임베딩이므로, ORTModelForFeatureExtraction 로 로딩함
model = ORTModelForFeatureExtraction.from_pretrained(model_path)

print(model)


In [ ]:
# 테스트 해봄
text = '난 널 사랑해'
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)
print(outputs)

output = outputs.last_hidden_state
print(output.shape)
#print(output[0][5])
onnx_embedding = output[0][5]
print(onnx_embedding)

In [ ]:
#sentence bert 원래 모델 로딩 
from sentence_transformers import SentenceTransformer, util
embedder = SentenceTransformer(model_path, device='cpu')
print(embedder)

In [42]:
corpus = [
        '난 널 사랑해', 
        '난 매우 행복해', 
        '날씨가 좋다',
        '안 그래도 되는데 뭐. 괜찮아.',
        '내일 저녁까지 보수 공사가 끝날 것으로 예상합니다.',
        '감사합니다. 후회 없는 결정이 될 겁니다.',
        '그러면 오전 10시에 보도록 하죠. 내일 뵙겠습니다.',
        '프린트 카트리지가 다 떨어졌습니다. 더 주문할까요?',
        '내일 비가 온다',
        '오늘은 운동을 해야지',
        '내일은 다른팀과 축구경기가 있다',
        '주말에는 등산을 갈 예정이다'
        ]

In [43]:
# ** 멀티로 한번에 tokenizer 할때는 반드시 padding=True 해야 함.(그래야 최대 길이 token에 맞춰서 padding 됨)
corpus_inputs = tokenizer(corpus, 
                 add_special_tokens=True, 
                 truncation=True, 
                 padding=True,   
                 max_length=128, 
                 return_tensors="pt")
print(corpus_inputs)
print(type(corpus_inputs))

{'input_ids': tensor([[   101,   8984,   9006, 119730,  14523,    102,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0],
        [   101,   8984,  42608, 121717,  14523,    102,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0],
        [   101, 123665,  11287,   9685,  11903,    102,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0],
        [   101,   9521,   8924,  37388,  12092,  54780,  28911,   9303,    119,
           8904, 119250,  16985,    119,    102,      0,      0,      0,      0],
        [   101, 129345, 121492,  18382, 120426, 120053,  11287,   8977,  41919,
          23925, 120364,  33188,  48345,    119,    102,      0,      0,      0],
        [   101, 121467,  33188,  48345,    119, 127364,  40364, 119583,  10739,
           9100, 136592,    119,    102,      0,      0,      0,      0,      0],
        

In [55]:
# onnx 양자화 모델 문장 유사도 출력 
import time

start = time.time()

# onnx 양자화 모델 문장 유사도 구함
outputs = model(**corpus_inputs)
last_hidden_state = outputs.last_hidden_state

print(last_hidden_state.shape)

# 첫번째 문장을 query로 지정함
in_mean_sequence = torch.mean(last_hidden_state[0], dim=0)

out_dict = {}
# for문을 돌면서 유사도 비교
for idx, hidden in enumerate(last_hidden_state):
    out_mean_sequence = torch.mean(hidden, dim=0)
    simul_score = util.pytorch_cos_sim(in_mean_sequence, out_mean_sequence)[0]
    #print("input vs {:d} 유사도:{}".format(idx, simul_score))
    
     # 사전 key로 순번으로 하고, 유사도를 저장함
    key = str(idx+1)
    out_dict[key] = simul_score
    #print("input vs {:d} 유사도:{}".format(idx, simul_score))
    
#print(out_dict)

# 사전 정렬(value(유사도)로 reverse=True 하여 내림차순으로 정렬함)
sorted_dict = sorted(out_dict.items(), key = lambda item: item[1], reverse = True)
#print(sorted_dict)

# 내립차순으로 정렬된 사전출력 
logger.info(f'---------------------------------------------------------')
logger.info(f'양자화/최적화된 모델 : {model_checkpoint}')
logger.info(f'---------------------------------------------------------')
for count in (sorted_dict):
    value = count[1].tolist() # count[1]은 2차원 tensor이므로 이를 list로 변환
    index = int(count[0])
    logger.info('{}, 유사도:{}'.format(corpus[index-1], value[0]))

logger.info(f'---------------------------------------------------------')
logger.info(f'=== 유사도 처리시간: {time.time() - start:.3f} 초 ===')
logger.info(f'-END-\n')

2022-05-26 15:01:23,142 - distilbertnlitest - INFO - ---------------------------------------------------------
INFO:distilbertnlitest:---------------------------------------------------------
2022-05-26 15:01:23,144 - distilbertnlitest - INFO - 난 널 사랑해, 유사도:0.9999998807907104
INFO:distilbertnlitest:난 널 사랑해, 유사도:0.9999998807907104
2022-05-26 15:01:23,145 - distilbertnlitest - INFO - 난 매우 행복해, 유사도:0.5999007225036621
INFO:distilbertnlitest:난 매우 행복해, 유사도:0.5999007225036621
2022-05-26 15:01:23,147 - distilbertnlitest - INFO - 날씨가 좋다, 유사도:0.38652727007865906
INFO:distilbertnlitest:날씨가 좋다, 유사도:0.38652727007865906
2022-05-26 15:01:23,148 - distilbertnlitest - INFO - 감사합니다. 후회 없는 결정이 될 겁니다., 유사도:0.3548429012298584
INFO:distilbertnlitest:감사합니다. 후회 없는 결정이 될 겁니다., 유사도:0.3548429012298584
2022-05-26 15:01:23,149 - distilbertnlitest - INFO - 오늘은 운동을 해야지, 유사도:0.3499101996421814
INFO:distilbertnlitest:오늘은 운동을 해야지, 유사도:0.3499101996421814
2022-05-26 15:01:23,150 - distilbertnlitest - INFO - 안 그래도 되는데 뭐. 

In [57]:
# sentence bert 문장 유사도 출력 
import time

start = time.time()

# sentence bert 문장 임베딩값 구함
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
print(corpus_embeddings.shape)
#print(corpus_embeddings)

# 첫번째 문장을 query로 지정함
in_mean_sequence = corpus_embeddings[0]

out_dict = {}
# for문을 돌면서 유사도 비교
for idx, hidden in enumerate(corpus_embeddings):
    out_mean_sequence = hidden
    simul_score = util.pytorch_cos_sim(in_mean_sequence, out_mean_sequence)[0]
    #print("input vs {:d} 유사도:{}".format(idx, simul_score))
    
     # 사전 key로 순번으로 하고, 유사도를 저장함
    key = str(idx+1)
    out_dict[key] = simul_score
    #print("input vs {:d} 유사도:{}".format(idx, simul_score))
    
#print(out_dict)

# 사전 정렬(value(유사도)로 reverse=True 하여 내림차순으로 정렬함)
sorted_dict = sorted(out_dict.items(), key = lambda item: item[1], reverse = True)
#print(sorted_dict)

# 내립차순으로 정렬된 사전출력 
logger.info(f'---------------------------------------------------------')
logger.info(f'sbert모델: {model_checkpoint}')
logger.info(f'---------------------------------------------------------')
for count in (sorted_dict):
    value = count[1].tolist() # count[1]은 2차원 tensor이므로 이를 list로 변환
    index = int(count[0])
    logger.info('{}, 유사도:{}'.format(corpus[index-1], value[0]))

logger.info(f'---------------------------------------------------------')
logger.info(f'=== 유사도 처리시간: {time.time() - start:.3f} 초 ===')
logger.info(f'-END-\n')

2022-05-26 15:19:43,320 - distilbertnlitest - INFO - ---------------------------------------------------------
INFO:distilbertnlitest:---------------------------------------------------------
2022-05-26 15:19:43,323 - distilbertnlitest - INFO - 난 널 사랑해, 유사도:1.000000238418579
INFO:distilbertnlitest:난 널 사랑해, 유사도:1.000000238418579
2022-05-26 15:19:43,325 - distilbertnlitest - INFO - 난 매우 행복해, 유사도:0.4053022861480713
INFO:distilbertnlitest:난 매우 행복해, 유사도:0.4053022861480713
2022-05-26 15:19:43,326 - distilbertnlitest - INFO - 날씨가 좋다, 유사도:0.20894736051559448
INFO:distilbertnlitest:날씨가 좋다, 유사도:0.20894736051559448
2022-05-26 15:19:43,328 - distilbertnlitest - INFO - 내일 비가 온다, 유사도:0.1295628547668457
INFO:distilbertnlitest:내일 비가 온다, 유사도:0.1295628547668457
2022-05-26 15:19:43,331 - distilbertnlitest - INFO - 감사합니다. 후회 없는 결정이 될 겁니다., 유사도:0.11919154971837997
INFO:distilbertnlitest:감사합니다. 후회 없는 결정이 될 겁니다., 유사도:0.11919154971837997
2022-05-26 15:19:43,333 - distilbertnlitest - INFO - 오늘은 운동을 해야지, 유사도:0.